<a href="https://www.kaggle.com/code/nigamshitij/goodreads-book-rating-data-analysis?scriptVersionId=193436850" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
# imports
import requests
from bs4 import BeautifulSoup

In [4]:
# request setup

url = "https://www.goodreads.com/shelf/show/fantasy"
container = ["title", "book_url", "author", "avg_rating", "ratings", "published_year"]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
 
response = requests.get(url, headers=headers)
html_content = response.text
# response = requests.get(url)
# print(response)
html_content = response.text
# print(html_content)
soup = BeautifulSoup(html_content, "html.parser")
# print(soup.prettify())

In [5]:
# other checks

total_items_info = soup.find("div", class_="mediumText")
print(total_items_info)

items = total_items_info.get_text().strip()
total_items = int(items.split()[-1].replace(",",""))
print(total_items)

<div class="mediumText">
<span class="smallText">
Showing 1-50 of 100,000
</span>
</div>
100000


In [6]:
# Basic Code to loop through - remove later

# temp fields
title = []
url_list = []
authors = []
avg_ratings = []
rating = []
year = []

# define genre
genre = "fantasy"

# define headers, url, other hyperparameters for request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
base_url = "https://www.goodreads.com/shelf/show/"+genre
items_per_page = 50
total_pages = (total_items + items_per_page - 1)
max_pages_to_scrape = 25

# iterate through pages
for page in range(1, min(max_pages_to_scrape, total_pages) + 1):
    # Construct the URL for the current page
    url = f"{base_url}?page={page}"
    response = requests.get(url, headers=headers).text
    soup = BeautifulSoup(response, "html.parser")
    book_elements = soup.find_all("div", "elementList")
    
    # Iterate through each book element on the current page
    for book_element in book_elements:
        # Use try-except blocks to handle potential errors if elements are missing
        try:
            # Extract book details
            book_title = book_element.find("a", "bookTitle").text
            book_url = "https://www.goodreads.com" + book_element.find("a", "bookTitle").get("href")
            author = book_element.find("a", "authorName").text
            rating_text = book_element.find("span", "greyText smallText").text.split()
            avg_rating = rating_text[2]
            ratings = rating_text[4]
            published_year = rating_text[-1] if len(rating_text) == 9 else ""
            
            # Append the extracted data to their respective lists
            title.append(book_title)
            url_list.append(book_url)
            authors.append(author)
            avg_ratings.append(avg_rating)
            rating.append(ratings)
            year.append(published_year)
        except AttributeError:
            # Handle the case where an element is not found
            print(f"Skipping a book on page {page} due to missing data.")

# store data from temp fields into dataframe          
good_reads = pd.DataFrame({
    "Title": title,
    "URL": url_list,
    "Authors": authors,
    "Avg Ratings": avg_ratings,
    "Rating": rating,
    "Published_year": year
})

# print head to check data
print(good_reads.head())

# store file into genre CSV
filename = "goodreads_" + genre + ".csv"
good_reads.to_csv(filename, index=False)

# print average rating
avg_rating = (good_reads["Avg Ratings"].astype(float).sum())/len(good_reads.index)
print(f"Average rating of {genre}: {avg_rating}")

Skipping a book on page 1 due to missing data.
Skipping a book on page 1 due to missing data.
Skipping a book on page 2 due to missing data.
Skipping a book on page 2 due to missing data.
Skipping a book on page 3 due to missing data.
Skipping a book on page 3 due to missing data.
Skipping a book on page 4 due to missing data.
Skipping a book on page 4 due to missing data.
Skipping a book on page 5 due to missing data.
Skipping a book on page 5 due to missing data.
Skipping a book on page 6 due to missing data.
Skipping a book on page 6 due to missing data.
Skipping a book on page 7 due to missing data.
Skipping a book on page 7 due to missing data.
Skipping a book on page 8 due to missing data.
Skipping a book on page 8 due to missing data.
Skipping a book on page 9 due to missing data.
Skipping a book on page 9 due to missing data.
Skipping a book on page 10 due to missing data.
Skipping a book on page 10 due to missing data.
Skipping a book on page 11 due to missing data.
Skipping a

In [7]:
good_reads.head()

,Title,URL,Authors,Avg Ratings,Rating,Published_year,Cover Image URL
0,Harry Potter and the Sorcerer’s Stone (Harry P...,https://www.goodreads.com/book/show/42844155-h...,J.K. Rowling,4.47,"10,318,696",1997,
1,Harry Potter and the Chamber of Secrets (Harry...,https://www.goodreads.com/book/show/15881.Harr...,J.K. Rowling,4.43,"4,038,252",1998,
2,Harry Potter and the Prisoner of Azkaban (Harr...,https://www.goodreads.com/book/show/5.Harry_Po...,J.K. Rowling,4.58,"4,314,999",1999,
3,"The Hobbit (The Lord of the Rings, #0)",https://www.goodreads.com/book/show/5907.The_H...,J.R.R. Tolkien,4.29,"4,087,286",1937,
4,Harry Potter and the Goblet of Fire (Harry Pot...,https://www.goodreads.com/book/show/6.Harry_Po...,J.K. Rowling,4.57,"3,795,465",2000,


In [8]:
# Extract All Genres - One time only, may need to rerun for non top level books

top_genre_url_1 = "https://www.goodreads.com/genres/list?filter=top-level"
top_genre_url_2 = "https://www.goodreads.com/genres/list?filter=top-level"

genres = []
genre_urls = []
# genre_books = []

for page in range(1, min(2, total_pages) + 1):
    # Construct the URL for the current page
    url = f"{top_genre_url_1}&page={page}"
    response = requests.get(url, headers=headers).text
    soup = BeautifulSoup(response, "html.parser")
    book_genres = soup.find_all("div", "shelfStat")
    
    # Iterate through each book element on the current page
    for book_genre in book_genres:
        # Use try-except blocks to handle potential errors if elements are missing
        try:
            # Extract book details
            genre = book_genre.find("a", "mediumText").text
            genre_url = "https://www.goodreads.com" + book_genre.find("a", "mediumText").get("href")
#             genre_books = book_genre.find("a", "mediumText").text
    
            # Append the extracted data to their respective lists
            genres.append(genre)
            genre_urls.append(genre_url)
#             genre_books.append(genre_books)
        except AttributeError:
            # Handle the case where an element is not found
            print(f"Skipping a genre on page {page} due to missing data.")
            
genres = pd.DataFrame({
    "Genre": genres,
    "URL": genre_urls,
#     "Total Books": genre_books
})

# save to CSV
genres.to_csv("goodreads_genres.csv", index=False)

url_test = "https://www.goodreads.com/shelf/show/"+genres['Genre'][0]
print(url_test)

https://www.goodreads.com/shelf/show/40k


In [9]:
genres.head()

,Genre,URL
0,40k,https://www.goodreads.com/genres/40k
1,adolescence,https://www.goodreads.com/genres/adolescence
2,adult,https://www.goodreads.com/genres/adult
3,aeroplanes,https://www.goodreads.com/genres/aeroplanes
4,amish,https://www.goodreads.com/genres/amish


In [22]:
# write function to loop through aforementioned function
import time
from requests.exceptions import RequestException

def scrape_goodreads_genre(genre, base_url, max_pages_to_scrape=25, max_retries=3, retry_delay=5):
    # temp fields
    books_data = []

    # define headers for request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }
    
    base_url = "https://www.goodreads.com/shelf/show/"+genre
    
    # iterate through pages
    for page in range(1, max_pages_to_scrape + 1):
        url = f"{base_url}?page={page}"
        retries = 0
        while retries < max_retries:
            try:
                response = requests.get(url, headers=headers, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")
                book_elements = soup.find_all("div", "elementList")
                
                if not book_elements:
                    print(f"No more books found for {genre} after page {page-1}")
                    return pd.DataFrame(books_data)
                
                for book_element in book_elements:
                    try:
                        book_title = book_element.find("a", "bookTitle").text.strip()
                        book_url = "https://www.goodreads.com" + book_element.find("a", "bookTitle").get("href")
                        author = book_element.find("a", "authorName").text.strip()
                        rating_text = book_element.find("span", "greyText smallText").text.split()
                        avg_rating = rating_text[2]
                        ratings = rating_text[4]
                        published_year = rating_text[-1] if len(rating_text) == 9 else ""
                
                        books_data.append({
                            "Genre": genre,
                            "Title": book_title,
                            "URL": book_url,
                            "Authors": author,
                            "Avg Ratings": avg_rating,
                            "Rating": ratings,
                            "Published_year": published_year
                        })
                    except AttributeError:
                        pass
                
                break  # If successful, break out of the retry loop
            
            except RequestException as e:
                retries += 1
                print(f"Error on page {page} of {genre}: {str(e)}. Retry {retries}/{max_retries}")
                if retries == max_retries:
                    print(f"Failed to retrieve page {page} for {genre} after {max_retries} attempts. Moving to next page.")
                    break
                time.sleep(retry_delay)
        
        time.sleep(1)  # Delay between successful page requests
    
    good_reads = pd.DataFrame(books_data)
    if not good_reads.empty:
        avg_rating = good_reads["Avg Ratings"].astype(float).mean()
        print(f"Average rating of {genre}: {avg_rating:.2f}")
    else:
        print(f"No data was scraped for {genre}")
    
    return good_reads

# Read the genres from the CSV file
genres_df = pd.read_csv("goodreads_genres.csv")

# Initialize an empty list to store DataFrames for each genre
all_genres_data = []

# Loop through genres and scrape data
for index, row in genres_df.iterrows():
    genre = row['Genre']
    url = row['URL']
    
    print(f"Scraping data for {genre} genre...")
    df = scrape_goodreads_genre(genre, url)
    all_genres_data.append(df)
    print(f"Finished scraping {genre}. DataFrame shape: {df.shape}")
    # print(df.head())
    print("\n" + "="*50 + "\n")

# Combine all genre DataFrames into a single DataFrame
combined_df = pd.concat(all_genres_data, ignore_index=True)

# Save the combined DataFrame to a CSV file
combined_df.to_csv("goodreads_all_genres.csv", index=False)

print("All genres have been scraped.")
print(f"Combined data from all genres has been saved to 'goodreads_all_genres.csv'.")
print(f"Total number of books scraped: {len(combined_df)}")

Scraping data for 40k genre...
Average rating of 40k: 4.15
Finished scraping 40k. DataFrame shape: (1250, 7)


Scraping data for adolescence genre...
Average rating of adolescence: 4.01
Finished scraping adolescence. DataFrame shape: (1250, 7)


Scraping data for adult genre...
Average rating of adult: 4.15
Finished scraping adult. DataFrame shape: (1250, 7)


Scraping data for aeroplanes genre...
Error on page 24 of aeroplanes: HTTPSConnectionPool(host='www.goodreads.com', port=443): Read timed out. (read timeout=10). Retry 1/3
Average rating of aeroplanes: 3.93
Finished scraping aeroplanes. DataFrame shape: (1250, 7)


Scraping data for amish genre...
Error on page 10 of amish: HTTPSConnectionPool(host='www.goodreads.com', port=443): Read timed out. (read timeout=10). Retry 1/3
Average rating of amish: 4.19
Finished scraping amish. DataFrame shape: (1250, 7)


Scraping data for animals genre...
Error on page 2 of animals: HTTPSConnectionPool(host='www.goodreads.com', port=443): Read 

In [15]:
genre_df.head()

,Title,URL,Authors,Avg_Ratings,Rating,Published_year,Genre
0,Harry Potter and the Sorcerer’s Stone (Harry P...,https://www.goodreads.com/book/show/42844155-h...,J.K. Rowling,4.47,"10,318,696",1997,"[[[[[...], [...], [...], [...], [...], [...], ..."
1,Harry Potter and the Chamber of Secrets (Harry...,https://www.goodreads.com/book/show/15881.Harr...,J.K. Rowling,4.43,"4,038,252",1998,"[[[[[...], [...], [...], [...], [...], [...], ..."
2,Harry Potter and the Prisoner of Azkaban (Harr...,https://www.goodreads.com/book/show/5.Harry_Po...,J.K. Rowling,4.58,"4,314,999",1999,"[[[[[...], [...], [...], [...], [...], [...], ..."
3,"The Hobbit (The Lord of the Rings, #0)",https://www.goodreads.com/book/show/5907.The_H...,J.R.R. Tolkien,4.29,"4,087,286",1937,"[[[[[...], [...], [...], [...], [...], [...], ..."
4,Harry Potter and the Goblet of Fire (Harry Pot...,https://www.goodreads.com/book/show/6.Harry_Po...,J.K. Rowling,4.57,"3,795,465",2000,"[[[[[...], [...], [...], [...], [...], [...], ..."
